In [18]:
import os
import json
import pandas as pd
import traceback

In [201]:
from langchain import PromptTemplate, LLMChain
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

In [191]:
from dotenv import load_dotenv
load_dotenv()

True

In [192]:
KEY = os.getenv('COHERE_API_TOKEN')

In [197]:
llm = ChatCohere(cohere_api_key="KEY",model_name="command-r-plus")

In [202]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in a number of ways: \n\n- **Automated Testing**: Langsmith can be used to automate various testing tasks. With its advanced natural language processing capabilities, Langsmith can understand and interpret test cases written in natural language. It can then use this understanding to automatically generate test scripts, reducing the time and effort required to create test cases manually. \n\n- **Natural Language Test Scripts**: Langsmith allows testers to write test scripts in natural language, making it easier for non-technical stakeholders to understand and contribute to the testing process. This can improve collaboration and communication between testers, developers, and business analysts. \n\n- **Test Case Generation**: Langsmith can analyze your requirements and generate test cases that cover different scenarios, including edge cases and negative tests. This helps ensure comprehensive test coverage and can identify potential issues

In [203]:
from langchain.chains import SequentialChain
import PyPDF2

In [204]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [298]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [299]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below and use it as a guide and make sure that the formate is same as RESPOSE_JSON. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [300]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [301]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [302]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [303]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [304]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [305]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [306]:
file_path = r"C:\Users\shail\Desktop\Projects\mcq_generator\data.txt"

In [307]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [308]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [309]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [310]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [322]:
quiz=response.get("quiz")

In [323]:
print(quiz)

Sure! Here are 5 multiple-choice questions based on the provided text:

```json
{
  "1": {
    "mcq": "What is the main focus of the scientific study of Biology?",
    "options": {
      "a": "The behavior of living organisms",
      "b": "The structure and function of cells",
      "c": "The evolution and diversity of life",
      "d": "The chemical processes in living beings"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "Which of the following is NOT a unifying theme in Biology?",
    "options": {
      "a": "Cellular structure",
      "b": "Energy processing in organisms",
      "c": "Environmental regulation in organisms",
      "d": "Chemical reactions in the atmosphere"
    },
    "correct": "d"
  },
  "3": {
    "mcq": "What is the role of genes in Biology?",
    "options": {
      "a": "They carry information for building and functioning of organisms",
      "b": "They determine the structure and behavior of an organism",
      "c": "They regulate the internal environment

In [319]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

AttributeError: 'str' object has no attribute 'items'